- I'll rapidly go over the changes i made to the agent and environment to accomodate the DQN approach
    - model taken from <a href="https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html">pytorch's site<a/>
    - the invalid action cost in the environment is now set to -0.01, for the following reason: in order to follow the convention to choose the maximum action-value, in the TSP context this would convert to the 1/reward (where the reward is the distance between nodes). A bigger distance results to a lower action value.

In [1]:
from deep_q.custom_env_deep_q import TSPEnv


class TSPDistCost(TSPEnv):
    # (...)

    def __init__(self, *args, **kwargs):
        self.N = 7

        self.invalid_action_cost = -0.01
    # (...)

KeyboardInterrupt: 

- This also happens in the step process, where the reward is again defined as the inverse

In [ ]:
    def _STEP(self, action):
        done = False

        #check if the action chosen is the starting one
        #if action == self.start_node and self.step_count<(self.N + 1):

        if self.visit_log[action] > 0:
            # Node already visited
            #changed for DQN
            reward = 1/self.invalid_action_cost
            done = True
        else:
            #changed for DQN
            if(self.current_node == action):
                reward = 0
            else:
                reward = 1 / self.distance_matrix[self.current_node, action]
            self.current_node = action
            self.visit_log[self.current_node] = 1

        self.state = self._update_state()
        # See if all nodes have been visited
        unique_visits = self.visit_log.sum()

        if unique_visits == self.N:
            done = True

        return self.state, reward, done, {}

# DQN
- The DQN is defined as follows, please tell me if it makes sense
    - it takes as input the state and outputs a tensor of action values, to determine the best action to take (in theory the one with the highest value). The index of the action_value corresponds to the action (node chosen) to make.
    - the initial epsilon is 0.2 here, but i changed it around up to 0.5 to see the

In [ ]:
import torch
from torch import nn

class DQN(nn.Module):
    def __init__(self, n_nodes):
        super(DQN, self).__init__()

        self.n_actions = n_nodes
        self.gamma = 0.99
        self.final_epsilon = 0.0001
        self.initial_epsilon = 0.2
        self.decay_epsilon = 7000
        self.num_iterations = 2000000
        self.replay_mem_size = 10000
        self.minibatch_size = 16
        #inplace=True means that it will modify the input directly, without allocating any additional output.
        self.relu1 = nn.RReLU(inplace=True)

        self.relu2 = nn.RReLU(inplace=True)

        self.relu3 = nn.RReLU(inplace=True)
        self.fc4   = nn.Linear(in_features=8, out_features=512)
        self.relu4 = nn.RReLU(inplace = True)
        self.fc5 = nn.Linear(in_features=512, out_features=n_nodes)

    def forward(self, x):
        floats = []
        for val in x:
            floats.append(val.double())
        #then convert from list to tensor
        tensFloats = torch.stack(floats)
        out = self.relu1(tensFloats)
        out = self.relu2(out)
        out = self.relu3(out)
        out = (out.float())
        out = self.fc4(out)
        out = self.relu4(out)
        out = self.fc5(out)

        return out